In [1]:
import pandas as pd

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings


In [ ]:
raw_doc= TextLoader("books_tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n")
doc= text_splitter.split_documents(raw_doc)

In [3]:
doc[0]

Document(metadata={'source': 'books_tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s detai

In [4]:
# Load BAAI embedding model with LangChain wrapper
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-base-en-v1.5")


C:\Users\shova\AppData\Local\Temp\ipykernel_16948\1340373745.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-base-en-v1.5")


In [ ]:
db_books = Chroma.from_documents(
    documents=doc,
    embedding=embedding_function,
    persist_directory="./db_books"  # Directory to persist the database
)

In [7]:
db_books = Chroma(
    persist_directory="./db_books",
    embedding_function=embedding_function
)

In [8]:
query= 'A book about a hopeless poor boy'
query_out= db_books.similarity_search(query, k=5)

In [9]:
query_out

[Document(id='1afc129f-8644-4ce9-ab57-888a003ed025', metadata={'source': 'books_tagged_description.txt'}, page_content='9780486431680 A modern classic about a penniless, unemployed young writer, this powerful, autobiographical novel paints an unforgettable portrait of a man driven to the edge of self-destruction by forces beyond his control.'),
 Document(id='9e42366c-307b-4d0d-89b8-8f96f728a76a', metadata={'source': 'books_tagged_description.txt'}, page_content='"9781586853334 There\'s this boy. Let\'s say he\'s somewhere between nine and thirteen years old or so. You\'d like to see this kid get creative. You\'d like to see him get some exercise. You\'d like to see him get out from in front of the television. And you\'d love for him to be motivated enough to find some stuff to do on his own. This boy NEEDS The Big Book of Boy Stuff! This thick, durable book includes fascinating chapters on gross stuff, magic, emergencies, fireworks, games, experiments, jokes, activities, insults, pets,

In [11]:
books= pd.read_csv("books_with_emotions.csv")

In [12]:
books[books['isbn13'] == int(query_out[0].page_content.split()[0].strip(' "'))]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,anger,disgust,fear,joy,sadness,surprise,neutral
2522,9780486431680,0486431681,Hunger,Knut Hamsun;George Egerton,Biography & Autobiography,http://books.google.com/books/content?id=b9goA...,"A modern classic about a penniless, unemployed...",2003.0,4.06,134.0,26016.0,Hunger,9780486431680 A modern classic about a pennile...,0.064134,0.104007,0.480943,0.040564,0.549477,0.359358,0.078766


In [13]:
def retrieve_recommendations(query: str, top_k: int = 10):
    """Retrieve book recommendations based on a query."""
    recs = db_books.similarity_search(query, k=15)

    books_list= []

    for i in range(0, len(recs)):
        books_list+= [int(recs[i].page_content.split()[0].strip(' "'))]

    books_recs = books[books['isbn13'].isin(books_list)]

    return books_recs.head(top_k)

In [14]:
retrieve_recommendations('A book for children about magic and adventure')

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,anger,disgust,fear,joy,sadness,surprise,neutral
146,9780060556501,0060556501,"The Lion, the Witch and the Wardrobe (picture ...",C. S. Lewis,Juvenile Fiction,http://books.google.com/books/content?id=FlSpp...,Narnia: A magical land full of wonder and exci...,2004.0,4.19,48.0,5012.0,"The Lion, the Witch and the Wardrobe (picture ...",9780060556501 Narnia: A magical land full of w...,0.500389,0.146838,0.247718,0.863718,0.944420,0.494298,0.621472
423,9780064410342,006441034X,Howl's Moving Castle,Diana Wynne Jones,Juvenile Fiction,http://books.google.com/books/content?id=f3RHP...,"In the land of Ingary, such things as spells, ...",2001.0,4.30,329.0,148957.0,Howl's Moving Castle,"9780064410342 In the land of Ingary, such thin...",0.173449,0.948491,0.980461,0.125308,0.914453,0.131563,0.620893
440,9780066238500,0066238501,The Chronicles of Narnia (adult),C. S. Lewis,Fiction,http://books.google.com/books/content?id=3VGkK...,"Journeys to the end of the world, fantastic cr...",2001.0,4.26,767.0,425445.0,The Chronicles of Narnia (adult),9780066238500 Journeys to the end of the world...,0.064134,0.104007,0.051363,0.487126,0.951270,0.111690,0.130764
779,9780141311883,0141311886,The Blue Sword,Robin McKinley,NaN,http://books.google.com/books/content?id=nDW2H...,"Harry, bored with her sheltered life in the re...",2001.0,4.23,272.0,286.0,The Blue Sword,"9780141311883 Harry, bored with her sheltered ...",0.064134,0.104007,0.728045,0.067011,0.751879,0.142349,0.125703
1476,9780345431912,034543191X,The King of Elfland's Daughter,Edward John Moreton Drax Plunkett Baron Dunsany,Fiction,http://books.google.com/books/content?id=LIoBA...,A young prince ventures into a mysterious fore...,1999.0,3.84,240.0,4718.0,The King of Elfland's Daughter,9780345431912 A young prince ventures into a m...,0.007677,0.007397,0.758380,0.006304,0.093334,0.005131,0.121776
1779,9780380789023,0380789027,Smoke and Mirrors,Neil Gaiman,Fiction,http://books.google.com/books/content?id=VsEtB...,"In the deft hands of Neil Gaiman, magic is no ...",2005.0,4.05,365.0,50702.0,Smoke and Mirrors: Short Fictions and Illusions,9780380789023 In the deft hands of Neil Gaiman...,0.064134,0.104007,0.986538,0.424260,0.775632,0.111690,0.228620
2116,9780439411875,0439411874,"The Girl, the Dragon, and the Wild Magic",Dave Luckett,Juvenile Fiction,http://books.google.com/books/content?id=1wpcG...,"Failing out of magic school, Rhianna meets a w...",2003.0,3.86,119.0,450.0,"The Girl, the Dragon, and the Wild Magic","9780439411875 Failing out of magic school, Rhi...",0.064134,0.104007,0.764360,0.040564,0.549477,0.111690,0.078766
2431,9780452260115,0452260116,Song of Solomon,Toni Morrison,Fiction,http://books.google.com/books/content?id=-FDBt...,This novel takes readers into a magical and ri...,1987.0,4.04,337.0,1647.0,Song of Solomon,9780452260115 This novel takes readers into a ...,0.081415,0.104007,0.051363,0.575674,0.847207,0.111690,0.108903
2634,9780552548229,0552548227,Spindle's End,Robin McKinley,Children's stories,http://books.google.com/books/content?id=W6Ev8...,"""The princess has been missing since she was a...",2003.0,3.80,464.0,184.0,Spindle's End,"9780552548229 ""The princess has been missing s...",0.083660,0.104007,0.661400,0.882416,0.847774,0.973009,0.753017
3187,9780688143923,068814392X,The Twelve Dancing Princesses,Marianna Mayer,Juvenile Fiction,http://books.google.com/books/content?id=03BIr...,"For generations, children have loved the encha...",1998.0,4.32,40.0,83.0,The Twelve Dancing Princesses,"9780688143923 For generations, children have l...",0.064134,0.104007,0.051363,0.907973,0.549477,0.111690,0.361117


In [15]:
retrieve_recommendations('A book about harry potter')

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,anger,disgust,fear,joy,sadness,surprise,neutral
423,9780064410342,006441034X,Howl's Moving Castle,Diana Wynne Jones,Juvenile Fiction,http://books.google.com/books/content?id=f3RHP...,"In the land of Ingary, such things as spells, ...",2001.0,4.30,329.0,148957.0,Howl's Moving Castle,"9780064410342 In the land of Ingary, such thin...",0.173449,0.948491,0.980461,0.125308,0.914453,0.131563,0.620893
779,9780141311883,0141311886,The Blue Sword,Robin McKinley,NaN,http://books.google.com/books/content?id=nDW2H...,"Harry, bored with her sheltered life in the re...",2001.0,4.23,272.0,286.0,The Blue Sword,"9780141311883 Harry, bored with her sheltered ...",0.064134,0.104007,0.728045,0.067011,0.751879,0.142349,0.125703
1247,9780312369347,0312369344,Beatrix Potter,Linda Lear,Biography & Autobiography,http://books.google.com/books/content?id=T9uTx...,"Peter Rabbit, Mr. McGregor, and many other Bea...",2007.0,4.03,584.0,1270.0,Beatrix Potter: A Life in Nature,"9780312369347 Peter Rabbit, Mr. McGregor, and ...",0.064134,0.104007,0.067638,0.743568,0.946370,0.253607,0.125684
2097,9780439064866,0439064864,Harry Potter and the Chamber of Secrets (Book 2),"Rowling, J.K.",Juvenile Fiction,http://books.google.com/books/content?id=ZTE0B...,"In Harry Potter and the Chamber of Secrets, th...",2015.0,4.41,341.0,2115562.0,Harry Potter and the Chamber of Secrets (Book 2),9780439064866 In Harry Potter and the Chamber ...,0.132693,0.332576,0.828690,0.058105,0.482938,0.195350,0.976697
2105,9780439314558,0439314550,Conversations with J.K. Rowling,Lindsey Fraser,Juvenile Nonfiction,http://books.google.com/books/content?id=Ly8dl...,From her birth in Chipping Sodbury near Bristo...,2001.0,3.99,96.0,4184.0,Conversations with J.K. Rowling,9780439314558 From her birth in Chipping Sodbu...,0.064134,0.104007,0.051363,0.727209,0.936846,0.111690,0.273506
2124,9780439554893,0439554896,Harry Potter and the Chamber of Secrets,J. K. Rowling;Mary GrandPre,Juvenile Fiction,http://books.google.com/books/content?id=h2Y-P...,When the Chamber of Secrets is opened again at...,2003.0,4.41,352.0,6267.0,Harry Potter and the Chamber of Secrets,9780439554893 When the Chamber of Secrets is o...,0.064134,0.104007,0.988214,0.040564,0.549477,0.111690,0.078766
2125,9780439554930,0439554934,Harry Potter and the Sorcerer's Stone (Book 1),"Rowling, J.K.",Juvenile Fiction,http://books.google.com/books/content?id=9jA0B...,"In Harry Potter and the Sorcerer's Stone, Harr...",2015.0,4.47,320.0,5629932.0,Harry Potter and the Sorcerer's Stone (Book 1),9780439554930 In Harry Potter and the Sorcerer...,0.066295,0.692797,0.896822,0.888196,0.903927,0.038230,0.170045
2134,9780439655484,043965548X,Harry Potter and the Prisoner of Azkaban (Book 3),"Rowling, J.K.",Juvenile Fiction,http://books.google.com/books/content?id=IZN5B...,"For twelve long years, the dread fortress of A...",2015.0,4.55,435.0,2149872.0,Harry Potter and the Prisoner of Azkaban (Book 3),"9780439655484 For twelve long years, the dread...",0.234615,0.626737,0.965734,0.420133,0.942617,0.945264,0.189432
2142,9780439785969,0439785960,Harry Potter and the Half-Blood Prince (Book 6),"Rowling, J.K.",Juvenile Fiction,http://books.google.com/books/content?id=QzI0B...,When Harry Potter and the Half-Blood Prince op...,2015.0,4.56,652.0,1944099.0,Harry Potter and the Half-Blood Prince (Book 6),9780439785969 When Harry Potter and the Half-B...,0.246915,0.301372,0.966404,0.506825,0.923649,0.952776,0.328497
2149,9780439827607,0439827604,The Harry Potter Collection,J. K. Rowling,Juvenile Fiction,http://books.google.com/books/content?id=v0z1P...,The first six years of Harry Potter magic are ...,2005.0,4.73,3342.0,27410.0,The Harry Potter Collection: The First Six Spe...,9780439827607 The first six years of Harry Pot...,0.064134,0.104007,0.051363,0.055592,0.870908,0.111690,0.078766
